In [1]:
import numpy as np
import sqlite3
from scipy.spatial import distance #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


In [2]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where "Huvudgrupp" = "Gröt"')
for row in rows:
        result.append(row)

db_contents = np.array(result)

conn.close()

## Prototypiska i sin kategori
Syftet är att ta en huvudgrupp och se vad mittpunkten i den kategorin är och sen räkna ut vilka livsmedel som ligger närmast mittpunkten.

Varje näringsvärde blir en dimension, och varje livsmedel en punkt som har ett värde i alla de dimensionerna.

Medelvärdet av alla livsmedel blir en punkt som man kan utgå från när man räknar avstånd till alla livsmedlen, och när man har tillgång till avstånden så kan man ordna alla livsmedlen efter avståndet till mittpunkten.

Det är första steget. 

Andra steget är att se om man verkligen behöver alla näringsvärden för att ändå få samma ordning mellan livsmedlen. Vilka kan man ta bort?

### Utgångspunkter

Från Livsmedelsverkets data har vi valt en huvudgrupp, som "Grönsaker". Alla dess data ligger i db_contents, och den variabeln har en shape som säger hur många rader (livsmedel) och kolumner (namn, nummer, näringsvärden, klassificering) den innehåller.

In [3]:
db_contents.shape

(16, 63)

Kolumn 0 är namnet, 1 är numret och sen börjar näringsvärdena. 

columns är variabeln som innehåller de värden som används i beräkningarna. Under står de kolumner som är möjliga att använda. 

Vill man ha fram betydelsena av kolumnerna kan man göra något i stil med

```
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
``` 

In [4]:
columns = np.array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]


dataset blir de kolumner som används. Från databasen brukar de komma som unicode, så det är lämpligt att konvertera till float.

In [5]:
dataset = db_contents[:,columns].astype(float)

Det är fantastiskt enkelt att räkna medelvärdet av varje kolumn med funktionen numpy.mean, som returnerar en array med lika många element som antal kolumner

Obs: axis behövs, för annars får man fel medelvärde...

In [6]:
means = np.mean(dataset, axis=0)

In [8]:
print(means, '\n', means.shape)

[  8.17187500e+01   1.32768750e+01   1.62500000e+00   2.61312500e+00
   1.45937500e+00   8.00162500e+01   0.00000000e+00   1.01062500e+00
   1.97375000e+00   2.48062500e+00   9.23750000e-01   8.62500000e+00
   7.05000000e-01   6.56250000e-02   3.06250000e-02] 
 (15,)


Det är läskigt lätt även att räkna avståndet mellan alla punkterna i dataset och medelvärdena:

(Det är något med medelvärdena som gör att man måste sätta en extra klammer runt, för att dimensionaliteten ska stämma...)

In [9]:
dist = distance.cdist(dataset, [means], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

*dist* innehåller sen varje rads avstånd från medelvärdet i alla kolumner.

In [10]:
dist

array([[  9.65037999],
       [  9.90711721],
       [ 16.30842908],
       [ 23.01765402],
       [ 23.28781031],
       [ 19.70277288],
       [  5.20878551],
       [ 21.54672531],
       [ 10.26108408],
       [ 17.54485705],
       [ 24.08684722],
       [ 74.91966395],
       [ 24.89919444],
       [ 23.42549592],
       [  8.5120721 ],
       [ 16.97021756]])

Jag har svårt att vänja mig vid det här med implicit sortering.

Först sorterar vi alltså värdena och får en "ordning", som vi sen använder som index för den array vi egentligen vill sortera...

In [11]:
order = np.argsort(dist.T[0])

In [12]:
order

array([ 6, 14,  0,  1,  8,  2, 15,  9,  5,  7,  3,  4, 13, 10, 12, 11])

Eftersom vi har vår ursprungliga lista med oss i *db_contents* så kan vi skriva ut vilka kolumner som helst, och nu med raderna i rätt ordning utifrån medelvärdet i huvudgruppen.

Här tog det ett tag för mig att förstå hur jag skulle se på resultatet. Att se de första som näringsmässigt "lika" var inte så svårt, men hur är det med de livsmedel som hamnar sist i listan?

Jag tror att man ska se det som en (mångdimensionell) "periferi" där de sista i listan förstås är långt från centrum, men inte alls inbördes "lika". De kan ha stort avstånd till mitten i väldigt skilda dimensioner. Det enda som förenar dem är att de är udda... :-)

In [13]:
db_contents[order,0]

array(['Molinogröt grahamsmjöl torkad frukt linfrö',
       'Havregrynsgröt kokt m mjölk', 'Risgrynsgröt m lättmjölk',
       'Risgrynsgröt', 'Mannagrynsgröt',
       'Gröt för barn fullkorn ätf berik', 'Gröt fullkorn ospec.',
       'Kruskagröt fullkorn russin havregryn vetekli',
       'Havregrynsgröt fullkorn', 'Kornmjölsgröt', 'Bovetegröt m russin',
       'Grahamsgröt', 'Havregrynsgröt fullkorn fiberhavregryn',
       'Rågmjölsgröt', 'Risifrutti light fett 1,5%', 'Risifrutti 6% fett'], dtype=object)

## Del två. Att jämföra ordningar...

Den här ordningen mellan livsmedel bygger på att man tar ett urval kolumner. Typiskt tar man så mycket information man har (= alla kolumner) för att få en mer relevant ordning.

Men kanske är det så att inte alla kolumner behövs för att ge den här ordningen? Vilka kolumner skulle man kunna ta bort?

Det enklaste är att kolla om ordningarna är identiska, så i nedanstående är det bara det som görs. 

Dags att summera ihop mycket av del 1 i en funktion. Utgå från db_contents som typ en global variabel. Skicka in kolumner till funktionen, så returneras en ordning mellan livsmedlen i db_contents/dataset

In [14]:
def get_order(columns):
    dataset = db_contents[:,columns].astype(float)
    means = np.mean(dataset, axis=0)
    dist = distance.cdist(dataset, [means], 'euclidean')
    order = np.argsort(dist.T[0])
    return order

Vi ska jämföra ursprungsordningen (order_0) med alla ordningar som uppkommer när man tar bort en kolumn. Vi kallar underordningarna för order_1 och löper igenom dem en i taget.

In [15]:
columns_0 = columns
order_0 = get_order(columns)

In [16]:
order_0

array([ 6, 14,  0,  1,  8,  2, 15,  9,  5,  7,  3,  4, 13, 10, 12, 11])

Här är ett exempel på vad som händer när man tar bort en kolumn:

In [17]:
columns_1 = np.delete(columns,4)

In [18]:
columns_1

array([ 2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18])

In [19]:
order_1 = get_order(columns_1)

In [20]:
x = np.array_equal(order_0, order_1)

In [21]:
x

True

In [22]:
columns_1

array([ 2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18])

Om vi tar en uppsättning kolumner behöver vi loopa igenom alla uppsättningar som saknar en kolumn. Något i den här stilen:

In [23]:
size = columns.size
print(size)
a = np.empty([size,size-1],dtype=int)
#print(a)
for index, item in enumerate(columns):
    a[index]=np.delete(columns,index)
    print(a[index])

15
[ 4  5  6  7  8  9 10 11 12 13 14 16 17 18]
[ 2  5  6  7  8  9 10 11 12 13 14 16 17 18]
[ 2  4  6  7  8  9 10 11 12 13 14 16 17 18]
[ 2  4  5  7  8  9 10 11 12 13 14 16 17 18]
[ 2  4  5  6  8  9 10 11 12 13 14 16 17 18]
[ 2  4  5  6  7  9 10 11 12 13 14 16 17 18]
[ 2  4  5  6  7  8 10 11 12 13 14 16 17 18]
[ 2  4  5  6  7  8  9 11 12 13 14 16 17 18]
[ 2  4  5  6  7  8  9 10 12 13 14 16 17 18]
[ 2  4  5  6  7  8  9 10 11 13 14 16 17 18]
[ 2  4  5  6  7  8  9 10 11 12 14 16 17 18]
[ 2  4  5  6  7  8  9 10 11 12 13 16 17 18]
[ 2  4  5  6  7  8  9 10 11 12 13 14 17 18]
[ 2  4  5  6  7  8  9 10 11 12 13 14 16 18]
[ 2  4  5  6  7  8  9 10 11 12 13 14 16 17]


Om vi ska göra en funktion av det hela och få ut en array av de överflödiga kolumnerna:

In [24]:
def remove_dimensions (columns):
    columns_1 = np.empty([columns.size,columns.size-1],dtype=int)
    superfluous = []
    #print(a)
    for index, item in enumerate(columns):
        columns_1[index]=np.delete(columns,index)
        if np.array_equal(get_order(columns), get_order(columns_1[index])):
            superfluous.append(item)
    return superfluous

In [25]:
print (columns_0)
superfluous = remove_dimensions (columns_0)
superfluous

[ 2  4  5  6  7  8  9 10 11 12 13 14 16 17 18]


[5, 6, 7, 8, 9, 10, 12, 13, 16, 17, 18]